In [7]:
#Import libraries
import torch 
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
import tensorflow as tf
AUTO = tf.data.experimental.AUTOTUNE
%matplotlib inline

In [8]:
def load_data(root_path, dir, batch_size):
    transform_dict = {'norm' : transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])}
    
    data = datasets.ImageFolder(root=root_path + dir, transform=transform_dict['norm'])
    
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, num_workers=2)
    
    return data_loader

In [9]:
# numpy and matplotlib defaults
np.set_printoptions(threshold=15, linewidth=80)

def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case, these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is the case for test data)
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)
    
def display_batch_of_images(databatch, predictions=None):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
        
    # auto-squaring: this will drop data that does not fit into square or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_flower(image, title, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(CLASSES)))
    ax.set_xticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(CLASSES)))
    ax.set_yticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()
    
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [10]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)  # image format uint8 [0,255]
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_saturation(image, 0, 2)
    return image, label   

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    #dataset = dataset.map(data_augment)
    #dataset = dataset.repeat() # the training dataset must repeat for several epochs
    #dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(CLASSES)))
    ax.set_xticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(CLASSES)))
    ax.set_yticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()

IMAGE_SIZE = [224, 224]
BATCH_SIZE = 16 

PATH_SELECT = { # available image sizes
    192: '../input/dataset' + '/tfrecords-jpeg-192x192',
    224: '../input/dataset' + '/tfrecords-jpeg-224x224',
    331: '../input/dataset' + '/tfrecords-jpeg-331x331',
    512: '../input/dataset' + '/tfrecords-jpeg-512x512'
}

PATH = PATH_SELECT[224]

TRAINING_FILENAMES = tf.io.gfile.glob(PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(PATH + '/test/*.tfrec')

dataset = get_training_dataset()
images = []
labels = []

for img, lbl in dataset:
    images.append(img)
    labels.append(lbl)
    
img_list=[]
for i in range(images.__len__()):
    for j in range(images[i].__len__()):
        img_list.append(images[i][j,:,:,:])
        
lab_list = list()
for label in labels:
    x = label.numpy().tolist()
    lab_list+=x
    


2022-02-11 13:23:11.833835: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-11 13:23:11.834836: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-11 13:23:11.835469: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-11 13:23:11.837284: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [11]:
import os
from PIL import Image

for i in range(len(lab_list)):
    img = np.array(img_list[i]).astype(np.uint8)
    lbl = lab_list[i]
    lbl = str(lbl)
    if not os.path.exists("../output/kaggle/working/dataset224/train/"+lbl):
        os.makedirs("../output/kaggle/working/dataset224/train/"+lbl)
    im = Image.fromarray(img)
    im.save("../output/kaggle/working/dataset224/train/"+lbl+"/"+str(i)+".jpeg")

dataset = get_validation_dataset()
val_images = []
val_labels = []

for img, lbl in dataset:
    val_images.append(img)
    val_labels.append(lbl)
    
val_img_list=[]
for i in range(val_images.__len__()):
    for j in range(val_images[i].__len__()):
        img_list.append(val_images[i][j,:,:,:])
        
val_lab_list = list()
for label in val_labels:
    x = label.numpy().tolist()
    val_lab_list+=x
    
for i in range(len(lab_list)):
    img = np.array(img_list[i]).astype(np.uint8)
    lbl = lab_list[i]
    lbl = str(lbl)
    if not os.path.exists("../output/kaggle/working/dataset224/val/"+lbl):
        os.makedirs("../output/kaggle/working/dataset224/val/"+lbl)
    im = Image.fromarray(img)
    im.save("../output/kaggle/working/dataset224/val/"+lbl+"/"+str(i)+".jpeg")

In [12]:
#define trainloader and test loader
trainloader = load_data('../output/kaggle/working/dataset224', '/train', 64)
testloader = load_data('../output/kaggle/working/dataset224', '/val', 64)


## Simple Model

In [13]:
#Define the model architecture

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        #Declare all the layers for extraction
        self.features = nn.Sequential(nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(4),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
          # Defining another 2D convolution layer
          nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(4),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2)
      )
        
        #Declare all layers for classification
        self.classifier = nn.Sequential(nn.Linear(56 * 56 * 4, 104))
        
    def forward(self, x):
        #Apply feature extractor to x
        x = self.features(x)
        
        #Squueze the 3 spatial dimensions into one
        x = x.view(x.size(0), -1)
        
        #Classify
        x = self.classifier(x)
        
        return x
        

In [16]:
#Define the model
model = Net()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

#Define the loss function
criterion = nn.CrossEntropyLoss()

# checking if GPU is available
#if torch.cuda.is_available():
 #   model = model.cuda()
  #  criterion = criterion.cuda()
    
print(model)

Net(
  (features): Sequential(
    (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=12544, out_features=104, bias=True)
  )
)


In [17]:
#Train the model for 10 epochs
for epoch in range(10):
    running_loss = 0
    for i, data in enumerate(trainloader, 0):
        
        
        #Get inputs
        inputs, labels = data
        
        if torch.cuda.is_available():
          inputs = inputs.cuda()
          labels = labels.cuda()
        
        #zero the parameter gradients
        optimizer.zero_grad()
        
        #Forward + backward + optimize
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
    
        
    print("Epoch {} - Training loss: {}".format(epoch+1, running_loss/len(trainloader)))
    print(output)
  

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [18]:
from torchvision import models

In [45]:
def train_model(model, dataloaders, testloaders, criterion, optimizer, device, num_epochs=25, is_train=True):
    since = time.time()
    
    acc_history = []
    loss_history = []

    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)
            model.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            # backward
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)

        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        if epoch_acc > best_acc:
            best_acc = epoch_acc
        elif epoch_acc < best_acc:
            break

        acc_history.append(epoch_acc.item())
        loss_history.append(epoch_loss)
        
        if not os.path.exists("../output/kaggle/working/temp"):
            os.makedirs("../output/kaggle/working/temp")
        
        torch.save(model.state_dict(), os.path.join('../output/kaggle/working/temp', '{0:0=2d}.pth'.format(epoch)))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Acc: {:4f}'.format(best_acc))
    
   
    
    return acc_history, loss_history, model

In [19]:
alexnet = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [27]:
transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) 

In [21]:
alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [22]:
#update second classifier
alexnet.classifier[4] = nn.Linear(4096, 1024)

#update output layer
alexnet.classifier[6] = nn.Linear(1024, 104)

In [23]:
alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [41]:
#Loss
criterion = nn.CrossEntropyLoss()

#optimizer
#optimizer = optim.SGD(alexnet.parameters(), lr = 0.001, momentum = 0.9)

In [42]:
# Here we only want to update the gradient for the classifier layer that we initialized.
params_to_update = []
for name,param in alexnet.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
            
optimizer = optim.SGD(params_to_update, lr = 0.001, momentum = 0.9)

	 features.0.weight
	 features.0.bias
	 features.3.weight
	 features.3.bias
	 features.6.weight
	 features.6.bias
	 features.8.weight
	 features.8.bias
	 features.10.weight
	 features.10.bias
	 classifier.1.weight
	 classifier.1.bias
	 classifier.4.weight
	 classifier.4.bias
	 classifier.6.weight
	 classifier.6.bias


In [63]:
#define trainloader and test loader
alex_trainloader = load_data('../output/kaggle/working/dataset224', '/train', 64)
alex_testloader = load_data('../output/kaggle/working/dataset224', '/val', 64)

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [46]:
train_model(alexnet, alex_trainloader, alex_testloader,criterion, optimizer, "cpu", num_epochs=10)

Epoch 0/9
----------
Loss: 0.2187 Acc: 0.9360

Epoch 1/9
----------
Loss: 0.1170 Acc: 0.9682

Epoch 2/9
----------
Loss: 0.0804 Acc: 0.9790

Epoch 3/9
----------
Loss: 0.0224 Acc: 0.9953

Epoch 4/9
----------
Loss: 0.0114 Acc: 0.9978

Epoch 5/9
----------
Loss: 0.0023 Acc: 0.9999

Epoch 6/9
----------
Loss: 0.0012 Acc: 0.9999

Epoch 7/9
----------
Loss: 0.0008 Acc: 1.0000

Epoch 8/9
----------
Loss: 0.0007 Acc: 1.0000

Epoch 9/9
----------
Loss: 0.0005 Acc: 1.0000

Training complete in 98m 10s
Best Acc: 1.000000


([0.9360150552811103,
  0.9681643534854544,
  0.978985336783502,
  0.9952952246530228,
  0.9978044381714106,
  0.9999215870775504,
  0.9999215870775504,
  1.0,
  1.0,
  1.0],
 [0.21870696648405008,
  0.11698300498757168,
  0.08044734101033844,
  0.022360630544272914,
  0.011361772172486878,
  0.0023292207765843688,
  0.0012002615980524415,
  0.0008452186057990594,
  0.0006666940712019044,
  0.0005476485426589506],
 AlexNet(
   (features): Sequential(
     (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
     (1): ReLU(inplace=True)
     (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
     (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
     (4): ReLU(inplace=True)
     (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
     (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (7): ReLU(inplace=True)
     (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1

In [ ]:
from sklearn.metrics import confusion_matrix

alexnet.load_state_dict(torch.load('../output/kaggle/working/temp/09.pth'))

predictions=[]
lbls=[]
for i, data in enumerate(alex_testloader, 0):
    #Get inputs
    inputs, labels = data
    outputs = alexnet(inputs)
    _, preds = torch.max(outputs, 1)
    
    predictions.append(preds)
    lbls.append(labels)

In [74]:
lbls = np.asarray(lbls).flatten()
print(lbls)

[tensor([ 44,  48,   2,  72,  50,  51,  68,  47,  52,  57,   6,  53,   6,  48,
          46,  40,  44,  75,  38,  25, 100,  41,  70,  22,  74,  68,  16,  15,
          34, 103,  77,  46,  75,   9,   9,  44,  38,  39,   9,  44,   6,  63,
          74, 100,  11,  77,  82,  77,  54,  68,  50,  85,  69,  99,  21,  94,
          68, 100,  47,   2,   8,  48,  80,  72])
 tensor([ 85,  48,  98,  58,  55,  68,   0,  94,  65,  37,   6,  45,   6,   8,
          77,  97,  75,   6,  94,   5,  75,  41,  74,  25,  45,  26,  25,   6,
          90,  68,  48,  49,  99,  61,  46,  11,  38,  87,   5,  68,  51,  78,
          53,  47,  69,   3, 100,  47,  48,   3,  48,  38,   6,  25,  56,  68,
          72,   4,  38,   4,  50,   0,  38,  53])
 tensor([ 69,  50,  75,  48,  68,  99,  10,  38,  83,  58,  41,  41,  18,  73,
          38,  96,  90,  26,  77,  80,  10,  97,  38,  97,  83, 100,  34,  44,
          44,  85,  69,  33,  89,  23,  77,  75,  77,  68,  69,  95,   0,  28,
          28,  10,   0,   4,  4

In [73]:
cm = confusion_matrix(lbls, predictions)
score=torch.sum(predictions == lbls)
precision=np.diag(cm) / np.sum(cm, axis=0)
recall=np.diag(cm) / np.sum(cm, axis=1)

display_confusion_matrix(cm, score, precision, recall)

ValueError: Found input variables with inconsistent numbers of samples: [200, 12753]